In [1]:
import data_utils

dataset = data_utils.get_dataset()

In [2]:
dataset = data_utils.clean_afec_dpto(dataset)
dataset = data_utils.clean_riesgo_vida(dataset)
dataset = data_utils.clean_cie_10(dataset)
dataset = data_utils.remove_features(dataset)
dataset = dataset.reset_index()
dataset = dataset.drop(['index'], axis = 1)

/Users/mrugeles/Documents/DataScience/Supersalud-PQRSD/data_utils.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataset['RIESGO_VIDA'] = np.where(dataset['RIESGO_VIDA'] == 'si', 1,0)


In [3]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values = '0', strategy="most_frequent")

zero_values = list(dataset.columns[dataset.eq('0').mean().mean() > 0])[0]
print(zero_values)
for feature in zero_values:
    print(feature)
    dataset[f'{feature}_is_missing'] = dataset[feature].apply(lambda value: 0 if value == '0' else 1)
    
dataset[zero_values] = imp.fit_transform(dataset[zero_values])

features_columns = [column for column in dataset.columns if '_is_missing' not in column]


['AFEC_DPTO' 'AFEC_EDADR' 'AFEC_EDUC' 'AFEC_GENERO' 'AFEC_GETNICO'
 'AFEC_MPIO' 'AFEC_PARENTESCO' 'AFEC_POBESPECIAL' 'AFEC_REGAFILIACION'
 'AFEC_TIPOPER' 'ALTO_COSTO' 'CIE_10' 'COD_MACROMOT' 'COD_MOTESP'
 'COD_MOTGEN' 'ENT_AMBITOIVC' 'ENT_COD_DEPTO' 'ENT_COD_MPIO' 'ENT_COD_SNS'
 'ENT_MPIO_DANE' 'ENT_NOMBRE' 'ENT_TIPOVIG' 'ENT_TIPOVIG_SNS'
 'IDPATOLOGIA_2' 'MES' 'PATOLOGIA_1' 'PATOLOGIA_TIPO' 'PET_COD_DEPTO'
 'PET_MPIO' 'PET_TIPOPER' 'PQR_CANAL' 'PQR_CLASE_SNS' 'PQR_TIPOATENCION'
 'PQR_TIPOPETICION' 'RIESGO_VIDA' 'TRIM']
AFEC_DPTO
AFEC_EDADR
AFEC_EDUC
AFEC_GENERO
AFEC_GETNICO
AFEC_MPIO
AFEC_PARENTESCO
AFEC_POBESPECIAL
AFEC_REGAFILIACION
AFEC_TIPOPER
ALTO_COSTO
CIE_10
COD_MACROMOT
COD_MOTESP
COD_MOTGEN
ENT_AMBITOIVC
ENT_COD_DEPTO
ENT_COD_MPIO
ENT_COD_SNS
ENT_MPIO_DANE
ENT_NOMBRE
ENT_TIPOVIG
ENT_TIPOVIG_SNS
IDPATOLOGIA_2
MES
PATOLOGIA_1
PATOLOGIA_TIPO
PET_COD_DEPTO
PET_MPIO
PET_TIPOPER
PQR_CANAL
PQR_CLASE_SNS
PQR_TIPOATENCION
PQR_TIPOPETICION
RIESGO_VIDA
TRIM


In [4]:
dtypes_df = dataset.dtypes.to_frame(name = 'dtype')
dtypes_df = dtypes_df[dtypes_df['dtype'] == 'object']
non_numeric_features = list(dtypes_df.index)

dataset[non_numeric_features] = dataset[non_numeric_features].applymap(str)

## Encoding dataset

In [5]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
for column in non_numeric_features:
    le.fit(dataset[column])
    dataset[column] = le.transform(dataset[column]) 

dataset.head()

,AFEC_DPTO,AFEC_EDADR,AFEC_EDUC,AFEC_GENERO,AFEC_GETNICO,AFEC_MPIO,AFEC_PARENTESCO,AFEC_POBESPECIAL,AFEC_REGAFILIACION,AFEC_TIPOPER,...,PATOLOGIA_TIPO_is_missing,PET_COD_DEPTO_is_missing,PET_MPIO_is_missing,PET_TIPOPER_is_missing,PQR_CANAL_is_missing,PQR_CLASE_SNS_is_missing,PQR_TIPOATENCION_is_missing,PQR_TIPOPETICION_is_missing,RIESGO_VIDA_is_missing,TRIM_is_missing
0,4,1,0,1,4,122,11,3,4,1,...,1,1,1,1,1,1,1,1,1,1
1,30,1,4,0,4,160,0,3,0,1,...,1,1,1,1,1,1,0,1,1,1
2,17,1,7,0,4,675,12,3,0,1,...,1,1,1,1,1,1,0,1,1,1
3,25,1,0,0,4,321,12,4,0,1,...,1,1,1,1,1,1,0,1,1,1
4,4,1,0,0,4,122,11,3,4,1,...,1,1,1,1,1,1,1,1,1,1


## Store processed dataset

In [6]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


dataset.to_csv("datasets/missing_state_impute_dataset.csv", index = False)